# AI Agent using Tool Calling in GPT-4o

In [1]:
import requests
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
client = OpenAI()

# Tools

In [25]:
def search_product(product_query):
    url = "https://real-time-amazon-data.p.rapidapi.com/search"

    querystring = {"query":product_query,
                   "page":"1",
                   "country":"US",
                   "sort_by":"RELEVANCE",
                   "product_condition":"ALL",
                   "is_prime":"false",
                   "deals_and_discounts":"NONE"}
    
    headers = {
    	"x-rapidapi-key": "<your rapid api key>",
    	"x-rapidapi-host": "real-time-amazon-data.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    return response.json()["data"]["products"][:3]

cart = []
def add_to_cart(product_id, quantity):
    global cart
    cart.append({"product_id": product_id, "quantity": quantity})
    return True

def summarize_cart():
    global cart
    return cart

def buy_product(payment_method):
    if payment_method == "COD":
        response = "Order confirmation sent to registered email"
    else:
        response = generate_payment_link(cart)
    return response

# Util function - not a tool to be provided to our agent
def generate_payment_link(cart):
    # TODO: actually implement this function
    return "sample.link.com"

In [4]:
known_actions = {
    "search_product": search_product,
    "add_to_cart": add_to_cart,
    "summarize_cart": summarize_cart,
    "buy_product": buy_product,  
}

# Tool descriptions for tool calling

In [5]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "search_product",
      "description": "Search for products on Amazon based on a query.",
      "strict": True,
      "parameters": {
        "type": "object",
        "required": [
          "product_query"
        ],
        "properties": {
          "product_query": {
            "type": "string",
            "description": "The query string for the product search"
          }
        },
        "additionalProperties": False
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "add_to_cart",
      "description": "Adds a product to the shopping cart with the specified quantity.",
      "strict": True,
      "parameters": {
        "type": "object",
        "required": [
          "product_id",
          "quantity"
        ],
        "properties": {
          "product_id": {
            "type": "string",
            "description": "Unique identifier for the product to be added to the cart."
          },
          "quantity": {
            "type": "number",
            "description": "The quantity of the product to add to the cart."
          }
        },
        "additionalProperties": False
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "summarize_cart",
      "description": "Returns the current contents of the cart",
      "strict": True,
      "parameters": {
        "type": "object",
        "properties": {},
        "additionalProperties": False
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "buy_product",
      "description": "Processes the purchase of a product using the specified payment method.",
      "strict": True,
      "parameters": {
        "type": "object",
        "required": [
          "payment_method"
        ],
        "properties": {
          "payment_method": {
            "type": "string",
            "description": "The method of payment to be used for the purchase.",
            "enum": [
              "COD",
              "CreditCard",
              "DebitCard",
              "UPI",
              "NetBanking"
            ]
          }
        },
        "additionalProperties": False
      }
    }
  }
]

In [10]:
result = client.chat.completions.create(
                        model="gpt-4o", 
                        messages=[{"role": "user", "content": "I want to search for a bowl for my dog"}],
                        tools=tools, tool_choice="auto")

In [11]:
result

ChatCompletion(id='chatcmpl-ApyU4aeP5jhsIHxfFXczzcu0WqLoQ', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pjbNnovKCviFi8pxp1kuBNY7', function=Function(arguments='{"product_query":"dog bowl"}', name='search_product'), type='function')]))], created=1736950592, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_50cad350e4', usage=CompletionUsage(completion_tokens=17, prompt_tokens=200, total_tokens=217, completion_tokens_details=CompletionTokensDetails(audio_tokens=0, reasoning_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [12]:
result.choices[0].message.content

In [13]:
result.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_pjbNnovKCviFi8pxp1kuBNY7', function=Function(arguments='{"product_query":"dog bowl"}', name='search_product'), type='function')]

# Agent loop with tool calling

In [14]:
def agent_loop(question_messages, max_turns=5):
    i = 0
    agent_loop_messages = question_messages

    while i < max_turns:
        i += 1
        result = client.chat.completions.create(
                        model="gpt-4o", 
                        messages=[{"role": "system", "content": "If no arg input is specified by user, ask user and do not assume."}] + agent_loop_messages,
                        tools=tools)
        
        if result.choices[0].finish_reason == "tool_calls":
            agent_loop_messages.append(result.choices[0].message)
            
            # Call all the tools that needs to be called
            for tool_call in result.choices[0].message.tool_calls:
                action = tool_call.function.name
                args = json.loads(tool_call.function.arguments)
            
                if action not in known_actions:
                    raise Exception("Unknown action: {}: {}".format(action, action_input))
                
                print("Calling tool: {} with input: {}".format(action, args))
                observation = known_actions[action](**args)
                print("Tool call {} with input: {} returned: {}".format(action, args, observation))
            
                agent_loop_messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": str(observation)
                })
        else:
            result = result.choices[0].message.content
            agent_loop_messages.append({"role": "assistant", "content": result})
            return result, agent_loop_messages
        
    return result, agent_loop_messages

# Chatting with our agent

In [15]:
messages = []

def chat(user_query):
    global messages
    messages.append({"role": "user", "content": user_query})

    print("======= Starting Agent Loop =======")
    response, agent_loop_history = agent_loop(messages)
    print("======= Ending Agent Loop =======\n\n")

    messages += agent_loop_history
    return response

In [16]:
result = chat("i want to buy a watch")
print(result)

======= Starting Agent Loop =======
======= Ending Agent Loop =======


Could you please provide more details about the watch you are looking for, such as brand, style, price range, or any specific features you prefer? This will help me find the best options for you.


In [17]:
result = chat("i want to buy a wrist watch gold in colour")
print(result)

======= Starting Agent Loop =======
Calling tool: search_product with input: {'product_query': 'gold wrist watch'}
Tool call search_product with input: {'product_query': 'gold wrist watch'} returned: [{'asin': 'B0D1TJFSKG', 'product_title': 'BOFAN Small Gold Watches for Women Easy Read Ladies Quartz Wrist Watch with Stainless Steel Expansion Band,Waterproof.', 'product_price': '$28.15', 'product_original_price': '$29.95', 'currency': 'USD', 'product_star_rating': '4.3', 'product_num_ratings': 262, 'product_url': 'https://www.amazon.com/dp/B0D1TJFSKG', 'product_photo': 'https://m.media-amazon.com/images/I/61Hm2WrH2sL._AC_UL960_FMwebp_QL65_.jpg', 'product_num_offers': 1, 'product_minimum_offer_price': '$28.15', 'is_best_seller': False, 'is_amazon_choice': False, 'is_prime': True, 'climate_pledge_friendly': False, 'sales_volume': '2K+ bought in past month', 'delivery': 'FREE delivery Tue, Jan 21 on $35 of items shipped by Amazon', 'has_variations': False}, {'asin': 'B075YYK9NL', 'product_

In [18]:
result = chat("third one please")
print(result)

======= Starting Agent Loop =======
Calling tool: add_to_cart with input: {'product_id': 'B0CFDKRBWY', 'quantity': 1}
Tool call add_to_cart with input: {'product_id': 'B0CFDKRBWY', 'quantity': 1} returned: True
======= Ending Agent Loop =======


The **TWOPTION Womens Waterproof Wrist Watch** has been added to your cart. Would you like to proceed to checkout, or do you need help with anything else?


In [19]:
result = chat("checkout now")
print(result)

======= Starting Agent Loop =======
======= Ending Agent Loop =======


Please choose your preferred payment method from the following options to proceed with the checkout:

- Cash on Delivery (COD)
- Credit Card
- Debit Card
- UPI
- Net Banking


In [20]:
result = chat("debit card")
print(result)

======= Starting Agent Loop =======
Calling tool: buy_product with input: {'payment_method': 'DebitCard'}
Tool call buy_product with input: {'payment_method': 'DebitCard'} returned: sample.link.com
======= Ending Agent Loop =======


Your purchase has been successfully processed using your Debit Card. Here is the [link](sample.link.com) to view your order details and track the shipment. 

If there's anything else you need, feel free to ask!


In [21]:
messages = []

In [22]:
cart = []

In [23]:
result = chat("i want to buy 2 umbrellas with cash")
print(result)

======= Starting Agent Loop =======
======= Ending Agent Loop =======


What specific type or brand of umbrella are you looking to purchase?


In [24]:
result = chat("i want to buy 2 umbrellas with cash of any brand")
print(result)

======= Starting Agent Loop =======
Calling tool: search_product with input: {'product_query': 'umbrella'}
Tool call search_product with input: {'product_query': 'umbrella'} returned: [{'asin': 'B0160HYB8S', 'product_title': 'Repel Umbrella Windproof Travel Umbrellas for Rain - Easy Auto Open Close, Durable &amp; Compact Umbrella, Strong Fiberglass Frame, Waterproof Canopy - Backpack, Purse, Portable Umbrella for Travel', 'product_price': '$26.98', 'product_original_price': '$29.99', 'currency': 'USD', 'product_star_rating': '4.5', 'product_num_ratings': 98913, 'product_url': 'https://www.amazon.com/dp/B0160HYB8S', 'product_photo': 'https://m.media-amazon.com/images/I/71CgcMEo59L._AC_UL960_FMwebp_QL65_.jpg', 'product_num_offers': 1, 'product_minimum_offer_price': '$26.98', 'is_best_seller': True, 'is_amazon_choice': False, 'is_prime': True, 'climate_pledge_friendly': False, 'sales_volume': '10K+ bought in past month', 'delivery': 'FREE delivery Tue, Jan 21 on $35 of items shipped by Am